In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt 
import seaborn as sns
import tensorflow as tf

plt.rcParams['font.size'] = 14

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

In [ ]:
datagen = ImageDataGenerator(brightness_range=[0.2,1.2],zoom_range=0.25)

In [ ]:
img_size = 150
batch_size = 32


# train_datagen = ImageDataGenerator(rescale=1./255)
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   brightness_range=[0.2,1.2],
                                   width_shift_range=0.1, 
                                   height_shift_range=0.1, 
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '../input/dogs-cats-images/dataset/training_set',  
        target_size=(img_size, img_size),  
        batch_size=batch_size,
        shuffle=True,
        class_mode='binary') 

validation_generator = test_datagen.flow_from_directory(
        '../input/dogs-cats-images/dataset/test_set',
        target_size=(img_size, img_size),
        batch_size=batch_size,
        shuffle=False,
        class_mode='binary')

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Layer, Flatten, Dense, Input

## Functional API
- We will be using this for many advanced architectures in the future
- Since the model has been redefined using the functional api, the functional api model will considered for the rest of the code

In [ ]:
input_layer = Input(shape=(150,150,3))
x = Conv2D(32, (3,3), input_shape=(150,150,3), activation='relu', padding='same')(input_layer)
x = MaxPooling2D(2,2)(x)
x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = MaxPooling2D(2,2)(x)
x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
x = MaxPooling2D(2,2)(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
output_layer = Dense(1, activation='sigmoid')(x)

model = Model(inputs=input_layer,outputs=output_layer)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
filepath= "model_cnn.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False)

In [ ]:
history = model.fit(train_generator,epochs=20,
                    validation_data=validation_generator,
                    callbacks=[checkpoint])

# Learning Curves
- If the difference between the validation loss and training loss is too big then your model is overfitting

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
model = tf.keras.models.load_model('/kaggle/working/model_cnn.h5')

In [ ]:
y_test = validation_generator.classes
y_pred = model.predict(validation_generator)
y_pred_probs = y_pred.copy()

In [ ]:
y_pred[y_pred>0.5] = 1
y_pred[y_pred<0.5] = 0

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

## Classification Report

In [ ]:
print(classification_report(y_test,y_pred,target_names = ['cats','dogs']))

## Confusion Matrix

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(confusion_matrix(y_test,y_pred),annot=True,fmt='.3g',xticklabels=['cats','dogs'],
            yticklabels=['cats','dogs'],cmap='Blues')
plt.show()

In [ ]:
filenames = validation_generator.filenames
data = pd.DataFrame()
data['filename'] = filenames
data['actual_class'] = y_test
data['predicted_class'] = y_pred
data['predicted_prob'] = y_pred_probs

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred_probs)
roc_auc = roc_auc_score(y_test, y_pred_probs)

In [ ]:
roc_auc

In [ ]:
plt.plot(fpr, tpr, color='blue', label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Random guessing')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) curve')
plt.legend(loc='lower right')
plt.show()